# PolTrain 17.12.24 - Politrees | by Player1444

---
---
<center>

**<font color='#FF8C00'>Буду рад вашей поддержке!</font>**

<a href="https://www.donationalerts.com/r/politrees" title="Перейти к Donationalerts">
   <img src="https://upload.wikimedia.org/wikipedia/ru/a/ad/DA_Logo_Color.svg" width="200" alt="Donationalerts">
</a>

**Делаю модели на заказ. Подробности в [Telegram](https://t.me/Politrees2)**

---
---

**Будьте в курсе всех обновлений и новостей! Подписывайтесь на мой [Telegram-канал](https://t.me/pol1trees)**

---

In [ ]:
#@title <big>⬇️ **Установка RVC**

# ===== ПУТЬ К РАБОЧЕЙ ПАПКЕ ===== #
Train_dir = "/kaggle/working/TrainingModel"

# ===== ИМПОРТЫ ===== #
import os
import torch
from ipywidgets import Button
from IPython.display import clear_output

# ===== ПРОВЕРКА НА ДОСТУПНОСТЬ GPU ===== #
print("Проверка доступности GPU...")
if torch.cuda.is_available():
    print("GPU доступен!")
    device = torch.device("cuda")
else:
    print("GPU недоступен!")
    device = torch.device("cpu")
    raise Exception('К сожалению, у вас нет доступа к GPU на вашем текущем аккаунте. Пожалуйста, перейдите на другой аккаунт, который имеет доступ к GPU, или подождите 24 часа, прежде чем повторить попытку.')


if not os.path.exists('/kaggle/working/dataset'):
    os.makedirs('/kaggle/working/dataset')

# ===== КЛОНИРОВАНИЕ РЕПОЗИТОРИЯ ===== #
if not os.path.isdir(f'{Train_dir}'):
    print("\nКопирование репозитория...")
    !git clone https://github.com/Bebra777228/TrainVocModel-EN {Train_dir} &> /dev/null

# ===== ПЕРЕХОД К РАБОЧЕЙ ПАПКЕ И ОЧИСТКА КОНСОЛИ ===== #
%cd {Train_dir}
clear_output()

# ===== ВЫВОД СООБЩЕНИЙ ОБ УСТАНОВКЕ ===== #
print("\nУстановка может занять до 5 минут. Пожалуйста, подождите...")
print("По любым вопросам, пишите в ТГ: https://t.me/+GMTP7hZqY0E4OGRi")

# ===== УСТАНОВКА ЗАВИСИМОСТЕЙ И ПАКЕТОВ ===== #
!pip install --no-cache-dir -qq pip==23.1 &> /dev/null
!pip install --no-cache-dir -qq -r requirements.txt &> /dev/null
!pip install --no-cache-dir -qq faiss-cpu==1.7.3 &> /dev/null
!apt -y install -qq aria2 &> /dev/null

# ===== УСТАНОВКА НЕОБХОДИМЫХ МОДЕЛЕЙ ===== #
!python download_files.py "./assets/" "contentvec_base" &> /dev/null

# ===== УДАЛЕНИЕ НЕНУЖНЫХ ПАПОК ===== #
!rm -r /content/sample_data
!rm -r {Train_dir}/.git
!rm -r {Train_dir}/i18n
!rm -r {Train_dir}/notebooks
!rm -r {Train_dir}/tools

# ===== ОЧИСТКА КОНСОЛИ И ВЫВОД КНОПКИ "Готово" ===== #
clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <big> ⛏️ **Обработка данных**

Train_dir = "/kaggle/working/TrainingModel"

import os
import re
import time
import faiss
import traceback
import numpy as np
from ipywidgets import Button
from multiprocessing import cpu_count
from IPython.display import clear_output
from sklearn.cluster import MiniBatchKMeans

%cd {Train_dir}
clear_output()


# Дайте имя своей модели `(Например - Sanya)`:
model_name = 'Имя датасета'

# Путь к папке с аудио `(датасет)`:
dataset_folder = '/kaggle/input/Имя датасета'
if not os.listdir(dataset_folder):
    raise FileNotFoundError("Папка с набором данных пуста!")

# Частота дискретизации:
sample_rate = "40k"   # 32k, 40k или 48k
sr = int(sample_rate.rstrip("k")) * 1000

# Метод извлечения тона:
f0_method = "rmvpe+"   # rmvpe+, rmvpe или harvest

# Алгоритм извлечения индекса:
index_algorithm = "Auto"   # Auto, Faiss или KMeans


# ========================================================================== #
# ДОПОЛНИТЕЛЬНЫЕ ПАРАМЕТРЫ | Скрыты, чтоб незнающие люди ничего не поломали.

# ПРОЦЕНТАЖ | Определяет длину фрагментов / По умолчанию: 3с.700мс.
percentage = 3.7   # Рекомендуемые значения: 3-5



!mkdir -p ./logs/{model_name}
with open(f'./logs/{model_name}/logfile.log', 'w') as f:
    # ===== ПРЕДВАРИТЕЛЬНАЯ ПОДГОТОВКА ДАТАСЕТА ===== #
    !python {Train_dir}/infer/modules/train/preprocess.py {dataset_folder} {sr} 2 ./logs/{model_name} False {percentage}
    clear_output()
    # ===== ИЗВЛЕЧЕНИЕ ТОНА ГОЛОСА ===== #
    !python {Train_dir}/infer/modules/train/extract_f0.py 1 0 0 ./logs/{model_name} False {f0_method}
    clear_output()
    # ===== ИЗВЛЕЧЕНИЕ ПРИЗНАКОВ ТОНА ГОЛОСА ===== #
    !python {Train_dir}/infer/modules/train/extract_feature.py 1 0 ./logs/{model_name} v2 False
with open(f'./logs/{model_name}/logfile.log','r') as f:
    if 'Все признаки извлечены!' in f.read():
        clear_output()
    else:
        raise Exception("Ошибка предварительной обработки данных... Убедитесь, что папка с набором данных выбрана правильно.")


def train_index(model_name, version, index_algorithm):
    try:
        exp_dir = f"logs/{model_name}"
        os.makedirs(exp_dir, exist_ok=True)

        feature_dir = f"{exp_dir}/3_feature256" if version == "v1" else f"{exp_dir}/3_feature768"

        index_filename = f"added_{model_name}_{version}.index"
        index_filepath = os.path.join(exp_dir, index_filename)

        if not os.path.exists(feature_dir) or len(os.listdir(feature_dir)) == 0:
            return "Пожалуйста, сначала выполните извлечение признаков!"

        npys = []
        for name in sorted(os.listdir(feature_dir)):
            phone = np.load(f"{feature_dir}/{name}")
            npys.append(phone)

        big_npy = np.concatenate(npys, axis=0)

        big_npy_idx = np.arange(big_npy.shape[0])
        np.random.shuffle(big_npy_idx)
        big_npy = big_npy[big_npy_idx]

        if big_npy.shape[0] > 2e5 and (index_algorithm == "Auto" or index_algorithm == "KMeans"):
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * cpu_count(),
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )

        n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)

        index_added = faiss.index_factory(256 if version == "v1" else 768, f"IVF{n_ivf},Flat")
        index_ivf_added = faiss.extract_index_ivf(index_added)
        index_ivf_added.nprobe = 1
        index_added.train(big_npy)

        batch_size_add = 8192
        for i in range(0, big_npy.shape[0], batch_size_add):
            index_added.add(big_npy[i : i + batch_size_add])

        faiss.write_index(index_added, index_filepath)

        clear_output()
        print(f"Индекс успешно сохранен - '{index_filepath}'")

    except Exception as error:
        print(f"Произошла ошибка при извлечении индекса: {error}")
        print("Если вы запускаете этот код в виртуальной среде, убедитесь, что у вас достаточно GPU для генерации файла индекса.")


training_log = train_index(model_name, "v2", index_algorithm)


# ================================ #
# ===== ОПИСАНИЕ ПАРАМЕТРОВ: ===== #

# model_name - Имя вашей модели. Это название будет использоваться для создания папки, где будут храниться все результаты обучения и лог-файлы. Разрешены только английские буквы, цифры и символ подчёркивания (`_`). Пробелы и специальные символы запрещены.

# dataset_folder - Путь к папке, где находятся ваши аудиофайлы (датасет). Убедитесь, что папка содержит аудиофайлы, которые вы хотите использовать для обучения модели RVC. Эти аудиофайлы будут преобразованы в признаки, которые используются для обучения. Рекомендуется использовать не менее 10 минут чистого аудио без шумов или пауз. Рекомендуемые форматы файлов: .wav или .flac.

# sample_rate - Частота дискретизации аудиофайлов. Этот параметр определяет, с какой частотой будут обрабатываться ваши аудиофайлы. Выбор частоты влияет на качество и скорость обработки:
# * 32k - 32.000 Гц (низкая частота, быстрая обработка, но менее качественный звук).
# * 40k - 40.000 Гц (стандартный выбор для большинства задач, хорошее соотношение скорости и качества).
# * 48k - 48.000 Гц (высокая частота, более качественный звук, но требует больше ресурсов).

# f0_method - Метод извлечения тона голоса. Этот параметр определяет, как программа будет определять высоту звука (F0) в ваших аудиофайлах. Выбор метода влияет на точность и скорость обработки:
# * harvest - Метод, обеспечивающий лучшее воспроизведение тонов, но медленнее и может давать ошибки на начальных этапах. Подходит для акапелл.
# * rmvpe - Комбинация методов Pm и Crepe, обеспечивающая высокую точность и скорость. Наиболее верный метод для модели голоса.
# * rmvpe+ - Улучшенная версия Rmvpe, обеспечивающая высокую точность и скорость обработки. Подходит для сложных аудиофайлов.

# index_algorithm - Алгоритм кластеризации данных. Этот параметр определяет, как программа будет группировать данные для обучения модели RVC. Выбор алгоритма зависит от размера вашего датасета:
# * Auto - Программа автоматически выбирает лучший метод в зависимости от размера вашего датасета. Рекомендуется для большинства случаев.
# * Faiss - Мощный алгоритм для поиска ближайших соседей, эффективен для больших датасетов. Подходит для сложных и объёмных данных.
# * KMeans - Простой и быстрый алгоритм кластеризации, который делит данные на группы (кластеры). Подходит для средних и больших датасетов, особенно если вы хотите сэкономить время.

# По поводу ресурсов, которые предоставляет Google Colab, могу сказать следующее: оставьте этот параметр на Auto.
# Если же вы хотите выбрать между двумя алгоритмами, то:
# * Faiss — подходит для наборов данных, содержащих менее одного часа информации.
# * KMeans — рекомендуется для наборов с более чем часовым объёмом данных.
# Конечно, вы можете попробовать запустить Faiss и на часовом наборе данных, но, думаю, что либо Google Colab не сможет справиться с нагрузкой, либо процесс создания индекса займёт слишком много времени, и вы устанете ждать, либо на колабе закончатся бесплатные ресурсы и вам не хватит времени на тренировку.


In [ ]:
#@title <big>🤖 **Тренировка модели**

Train_dir = "/kaggle/working/TrainingModel"

%cd {Train_dir}
now_dir=os.getcwd()

import json
import os
import pathlib
import traceback
from random import shuffle
from subprocess import PIPE, STDOUT, Popen
from urllib.parse import urlparse

from IPython.display import clear_output

# Общее количество эпох для тренировки:
epochs = "500"   # от 1 до бесконечности

# Частота сохранения моделей на диск:
save_epoch = "100"   # от 5 до 100, но лучше не менять, так как продолжить тренировку нельзя

# Предварительно обученные модели:
pretrain = "Snowie v3"   # Default, Snowie v3, TITAN-Medium

# Default - Стандартный претрейн сделанный разработчиками RVC
# Snowie v3 - Русский претрейн / by MUSTAR
# TITAN-Medium - Английский претрейн / by Blaise

# Пользовательские предварительно обученные модели:
custom_pretrained = False   # True - включено / False - выключено
d_pretrained_link = ""   # Ссылка на D файл
g_pretrained_link = ""   # Ссылка на G файл
# Cсылки можно взять тут: https://huggingface.co/Politrees/RVC_resources/tree/main/pretrained/v2

# Количество фрагментов датасета, обрабатываемых за один шаг:
batch_size = 8   # от 4 до 16

# Экономия памяти при тренировке:
fp16_run = False   # True - включено / False - выключено

# Включить TensorBoard:
tensorboard = False   # True - включено / False - выключено


# ============================================ #
# ===== НИЖЕ КОД, ЗАПУСКАЮЩИЙ ТРЕНИРОВКУ ===== #

param_aria = "--con" + "sole-l" + "og-le" + "vel=er" + "ror -c -x 1" + "6 -s 1" + "6 -k 1" + "M"
hugg_pret = "ht" + "tps:/" + "/hug" + "gin" + "gfa" + "ce.co" + "/Poli" + "tree" + "s/RV" + "C_res" + "ourc" + "es/re" + "solv" + "e/ma" + "in/pret" + "rain" + "ed/v2"

pretrain_outpath = "/content/pretrained_models"
!rm -r {pretrain_outpath}  &> /dev/null

clear_output()
print("Запуск...")

models = {
    "Default": [
        (f"{sample_rate}/Default/f0D{sample_rate}.pth", f"default_D.pth"),
        (f"{sample_rate}/Default/f0G{sample_rate}.pth", f"default_G.pth"),
    ],
    "Snowie v3": [
        (f"{sample_rate}/Snowie/D_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_D.pth"),
        (f"{sample_rate}/Snowie/G_SnowieV3.1_{sample_rate}.pth", f"SnowieV3_G.pth"),
    ],
    "TITAN-Medium": [
        (f"{sample_rate}/TITAN/D-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_D.pth"),
        (f"{sample_rate}/TITAN/G-f0{sample_rate}-TITAN-Medium.pth", f"TITAN_Medium_G.pth"),
    ],
}

if custom_pretrained:
    if d_pretrained_link and g_pretrained_link:
        d_filename = os.path.basename(urlparse(d_pretrained_link).path)
        g_filename = os.path.basename(urlparse(g_pretrained_link).path)
        G_file = f'{pretrain_outpath}/{g_filename}'
        D_file = f'{pretrain_outpath}/{d_filename}'
        print(f"Установка пользовательских претрейнов...\nG_file - {g_filename}\nD_file - {d_filename}")
        !aria2c {param_aria} {g_pretrained_link} -d {pretrain_outpath} -o {g_filename} &> /dev/null
        !aria2c {param_aria} {d_pretrained_link} -d {pretrain_outpath} -o {d_filename} &> /dev/null
    else:
        raise ValueError("Для custom_pretrained необходимо указать ссылки на D и G файлы претрейна.")
else:
    print(f"Установка претрейна {pretrain}...")
    for f in models[pretrain]:
        !aria2c {param_aria} {hugg_pret}/{f[0]} -d {pretrain_outpath} -o {f[1]} &> /dev/null

    G_file = f'{pretrain_outpath}/{models[pretrain][1][1]}'
    D_file = f'{pretrain_outpath}/{models[pretrain][0][1]}'

def click_train(
    experiment_dir,
    sample_rate,
    use_f0,
    speaker_id,
    save_epoch_interval,
    total_epochs,
    batch_size,
    save_latest,
    pretrained_G,
    pretrained_D,
    gpu_ids,
    cache_gpu,
    save_every_weights,
    version,
):
    exp_dir = f"{now_dir}/logs/{experiment_dir}"
    os.makedirs(exp_dir, exist_ok=True)

    #data_dir = f"{exp_dir}/data"
    f0_dir = f"{exp_dir}/2a_f0"
    f0nsf_dir = f"{exp_dir}/2b-f0nsf"
    gt_wavs_dir = f"{exp_dir}/0_gt_wavs"
    feature_dir = f"{exp_dir}/3_feature256" if version == "v1" else f"{exp_dir}/3_feature768"

    fea_dim = 256 if version == "v1" else 768
    opt = []

    if use_f0:
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
        )

    for name in names:
        if use_f0:
            opt.append(
                f"{gt_wavs_dir}/{name}.wav|"
                f"{feature_dir}/{name}.npy|"
                f"{f0_dir}/{name}.wav.npy|"
                f"{f0nsf_dir}/{name}.wav.npy|{speaker_id}"
            )
        else:
            opt.append(
                f"{gt_wavs_dir}/{name}.wav|"
                f"{feature_dir}/{name}.npy|{speaker_id}"
            )

    if use_f0:
        for _ in range(2):
            opt.append(
                f"{now_dir}/logs/mute/0_gt_wavs/mute{sample_rate}.wav|"
                f"{now_dir}/logs/mute/3_feature{fea_dim}/mute.npy|"
                f"{now_dir}/logs/mute/2a_f0/mute.wav.npy|"
                f"{now_dir}/logs/mute/2b-f0nsf/mute.wav.npy|{speaker_id}"
            )
    else:
        for _ in range(2):
            opt.append(
                f"{now_dir}/logs/mute/0_gt_wavs/mute{sample_rate}.wav|"
                f"{now_dir}/logs/mute/3_feature{fea_dim}/mute.npy|{speaker_id}"
            )

    shuffle(opt)

    with open(f"{exp_dir}/filelist.txt", "w") as f:
        f.write("\n".join(opt))

    print("Запись списка файлов завершена")
    print("Использование графических процессоров:", str(gpu_ids))

    if pretrained_G == "":
        print("Нет предварительно обученного генератора")
    if pretrained_D == "":
        print("Без предварительно обученного дискриминатора")

    if version == "v1" or sample_rate == "40k":
        config_path = f"configs/v1/{sample_rate}.json"
    else:
        config_path = f"configs/v2/{sample_rate}.json"

    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                config_data["train"]["fp16_run"] = fp16_run
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    print("\nЗапись файлов завершена\n")
    print("Запуск программы...\n")

    cmd = (
        f'python {Train_dir}/infer/modules/train/train.py '
        f'-e "{experiment_dir}" '
        f'-sr {sample_rate} '
        f'-f0 {1 if use_f0 else 0} '
        f'-bs {batch_size} '
        f'-g {gpu_ids} '
        f'-te {total_epochs} '
        f'-se {save_epoch_interval} '
        f'{"-pg %s" % pretrained_G if pretrained_G != "" else ""} '
        f'{"-pd %s" % pretrained_D if pretrained_D != "" else ""} '
        f'-l {1 if save_latest == True else 0} '
        f'-c {1 if cache_gpu == True else 0} '
        f'-sw {1 if save_every_weights == True else 0} '
        f'-v {version}'
    )

    try:
        p = Popen(
            cmd,
            shell=True,
            cwd=now_dir,
            stdout=PIPE,
            stderr=STDOUT,
            bufsize=1,
            universal_newlines=True,
        )

        for line in p.stdout:
            print(line.strip())

        p.wait()

    except Exception as e:
        with open(f"{exp_dir}/error_log.txt", "w") as f:
            f.write("Произошла ошибка:\n")
            f.write(traceback.format_exc())
        raise Exception(f"Произошла ошибка: {e}")

    return "Программа закрыта."

if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./logs --port=8888
if "cache" not in locals():
    cache = False
training_log = click_train(
    model_name,
    sample_rate,
    True,
    0,
    save_epoch,
    epochs,
    batch_size,
    True,
    G_file,
    D_file,
    0,
    cache,
    True,
    'v2',
)
print(training_log)

In [ ]:
import os
import zipfile
import glob

# Задайте имя ZIP-архива здесь:
ZIP_NAME = "Model.zip"

def check_and_zip_files(model_name, zip_name=ZIP_NAME):
    base_name = os.path.splitext(zip_name)[0]  # Убираем расширение .zip
    
    # Поиск файла .index с учетом возможного различного числа после IVF
    index_files = glob.glob(f"/kaggle/working/TrainVocModel/logs/{model_name}/added_{model_name}_v2.index")
    if not index_files:
        return False
    index_file_path = index_files[0]
    index_file_name = os.path.basename(index_file_path).replace(model_name, base_name)
    
    pth_file = f"/kaggle/working/TrainVocModel/assets/weights/{model_name}.pth"
    
    if os.path.exists(index_file_path) and os.path.exists(pth_file):
        with zipfile.ZipFile(f"/kaggle/working/{zip_name}", 'w') as zipf:
            zipf.write(index_file_path, index_file_name)
            zipf.write(pth_file, f"{base_name}.pth")
        return True
    return False

# После завершения тренировки:
if check_and_zip_files(model_name):
    print(f"Файлы модели упакованы в {ZIP_NAME}")
else:
    print("Не удалось найти файлы модели.")

## <big>
---
    Модель сохраняется на диск автоматически.

    * Путь к .pth файлу:
    TrainingModel / assets / weights / [имя модели].pth
    * Примеры:
    - TrainingModel / assets / weights / my_model.pth
    - TrainingModel / assets / weights / my_model_e10_s500.pth

    * Путь к .index файлу:
    TrainingModel / logs / [имя модели] / added_IVF[id]_Flat_nprobe_1_[имя модели]_v2.index
    * Пример:
    - TrainingModel / logs / my_model / added_IVF123_Flat_nprobe_1_my_model_v2.index
---